In [5]:
!curl -H "Accept: application/vnd.github+json" \
     -H "Authorization: Bearer API_KEY" \
     "https://api.github.com/search/users?q=location:Basel+followers:>10&per_page=100&page=1" > users1.json

!curl -H "Accept: application/vnd.github+json" \
     -H "Authorization: Bearer API_KEY" \
     "https://api.github.com/search/users?q=location:Basel+followers:>10&per_page=100&page=2" > users2.json

!curl -H "Accept: application/vnd.github+json" \
     -H "Authorization: Bearer API_KEY" \
     "https://api.github.com/search/users?q=location:Basel+followers:>10&per_page=100&page=3" > users3.json

!curl -H "Accept: application/vnd.github+json" \
     -H "Authorization: Bearer API_KEY" \
     "https://api.github.com/search/users?q=location:Basel+followers:>10&per_page=100&page=4" > users4.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   109  100   109    0     0    561      0 --:--:-- --:--:-- --:--:--   561
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   109  100   109    0     0    539      0 --:--:-- --:--:-- --:--:--   542
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   109  100   109    0     0    559      0 --:--:-- --:--:-- --:--:--   561
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   109  100   109    0     0    505      0 --:--:-- --:--:-- --:--:--   504


In [ ]:
import json
import csv

# Load JSON data from the file
ids=[]
with open('users1.json', 'r') as file:
    data = json.load(file)

for user in data['items']:
    ids.append(user['id'])

with open('users2.json', 'r') as file:
    data = json.load(file)

for user in data['items']:
    ids.append(user['id'])

with open('users3.json', 'r') as file:
    data = json.load(file)

for user in data['items']:
    ids.append(user['id'])

with open('users4.json', 'r') as file:
    data = json.load(file)

for user in data['items']:
    ids.append(user['id'])

len(ids)

In [ ]:
import requests
import csv
import time

# GitHub API token (replace with your own)
TOKEN = 'YOUR_API_KEY_HERE'

# Headers for the API request
headers = {
    'Accept': 'application/vnd.github+json',
    'Authorization': f'Bearer {TOKEN}'
}

# Output CSV file
csv_file = 'github_users.csv'

# Define the CSV file for writing
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    # Define the CSV writer
    writer = csv.writer(csvfile)

    # Write the header to the CSV
    writer.writerow([
        'login', 'name', 'company', 'location', 'email', 'hireable', 'bio',
        'public_repos', 'followers', 'following', 'created_at'
    ])

    # Loop through each user ID in the ids list
    i=0
    for user_id in ids:
        # Make the API request for the current user
        response = requests.get(f'https://api.github.com/user/{user_id}', headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            user_data = response.json()

            # Extract relevant fields, with fallback for missing data
            login = user_data.get('login', '')
            name = user_data.get('name', '')
            company = user_data.get('company', '').strip().lstrip('@').upper() if user_data.get('company') else ''
            location = user_data.get('location', '')
            email = user_data.get('email', '')
            hireable = user_data.get('hireable', '')
            bio = user_data.get('bio', '')
            public_repos = user_data.get('public_repos', 0)
            followers = user_data.get('followers', 0)
            following = user_data.get('following', 0)
            created_at = user_data.get('created_at', '')

            # Write the user's data to the CSV file
            writer.writerow([
                login, name, company, location, email, hireable, bio,
                public_repos, followers, following, created_at
            ])
            i+=1
            print(f"Fetched data for user ID {user_id}, progress {i}/{len(ids)}")
        else:
            print(f"Failed to fetch data for user ID {user_id}: {response.status_code}")

        # To avoid hitting rate limits, it's a good idea to add a short delay between requests
        time.sleep(1)

print(f"Data written to {csv_file}")


In [ ]:
import requests
import csv
import time

TOKEN = 'YOUR_API_KEY_HERE'
# Headers for the API request
headers = {
    'Accept': 'application/vnd.github+json',
    'Authorization': f'Bearer {TOKEN}'
}

# Output CSV file
csv_file = 'github_repos_limited.csv'

# Define the CSV file for writing
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    # Define the CSV writer
    writer = csv.writer(csvfile)

    # Write the header to the CSV
    writer.writerow([
        'login', 'full_name', 'created_at', 'stargazers_count', 'watchers_count',
        'language', 'has_projects', 'has_wiki', 'license_name'
    ])

    # Loop through each user ID in the ids list
    i=0
    for user_id in ids:
        i+=1
        page = 1
        total_repos_fetched = 0

        while total_repos_fetched < 500:
            # Make the API request for the current user repositories (up to 100 per page)
            response = requests.get(
                f'https://api.github.com/user/{user_id}/repos?sort=pushed&per_page=100&page={page}',
                headers=headers
            )

            # Check if the request was successful
            if response.status_code == 200:
                repos = response.json()

                # If there are no more repositories, break the loop
                if not repos:
                    break

                for repo in repos:
                    # Extract relevant repository details
                    full_name = repo.get('full_name', '')
                    created_at = repo.get('created_at', '')
                    stargazers_count = repo.get('stargazers_count', 0)
                    watchers_count = repo.get('watchers_count', 0)
                    language = repo.get('language', '')
                    has_projects = repo.get('has_projects', False)
                    has_wiki = repo.get('has_wiki', False)
                    license_info = repo.get('license')
                    license_name = license_info['key'] if license_info else ''

                    # Make an additional request to get user login since it's not in repo details
                    owner_response = requests.get(repo.get('owner', {}).get('url', ''), headers=headers)
                    if owner_response.status_code == 200:
                        login = owner_response.json().get('login', '')
                    else:
                        login = ''

                    # Write the repository data to the CSV
                    writer.writerow([
                        login, full_name, created_at, stargazers_count, watchers_count,
                        language, has_projects, has_wiki, license_name
                    ])

                    total_repos_fetched += 1

                    # Stop if we have fetched 500 repositories
                    if total_repos_fetched >= 500:
                        break

                print(f"Fetched {total_repos_fetched} repos for user ID {user_id} (page {page}) user {i}/{len(ids)}")
            else:
                print(f"Failed to fetch repos for user ID {user_id}: {response.status_code}")
                break

            # Move to the next page
            page += 1

            # Add a delay between API calls to avoid rate limits
            time.sleep(1)

print(f"Data written to {csv_file}")
